In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
target = train.TARGET.values.copy()
train.drop('TARGET', axis=1, inplace=True)

In [4]:
train.loc[:, 'n0'] = (train == 0).sum(axis=1)
test.loc[:, 'n0'] = (test == 0).sum(axis=1)

In [5]:
dlist = list()
for c in train.columns:
    if train[c].unique().shape[0] == 1:
        print(c)
        dlist.append(c)
train.drop(dlist, axis=1, inplace=True)
test.drop(dlist, axis=1, inplace=True)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var33_hace3
imp_trasp_var17_out_hace3
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var33_hace3
num_trasp_var17_out_hace3
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3


In [6]:
for cone in train.columns:
    for ctwo in train.columns:
        if cone not in train.columns:
            continue
        if ctwo not in train.columns:
            continue
        if cone != ctwo and np.all(train[cone] == train[ctwo]):
            print(cone, ctwo)
            train.drop(ctwo, axis=1, inplace=True)
            test.drop(ctwo, axis=1, inplace=True)

ind_var6_0 ind_var29_0
ind_var6 ind_var29
ind_var13_medio_0 ind_var13_medio
ind_var18_0 ind_var18
ind_var26_0 ind_var26
ind_var25_0 ind_var25
ind_var32_0 ind_var32
ind_var34_0 ind_var34
ind_var37_0 ind_var37
ind_var40 ind_var39
num_var6_0 num_var29_0
num_var6 num_var29
num_var13_medio_0 num_var13_medio
num_var18_0 num_var18
num_var26_0 num_var26
num_var25_0 num_var25
num_var32_0 num_var32
num_var34_0 num_var34
num_var37_0 num_var37
num_var40 num_var39
saldo_var6 saldo_var29
saldo_var13_medio saldo_medio_var13_medio_ult1
delta_imp_reemb_var13_1y3 delta_num_reemb_var13_1y3
delta_imp_reemb_var17_1y3 delta_num_reemb_var17_1y3
delta_imp_reemb_var33_1y3 delta_num_reemb_var33_1y3
delta_imp_trasp_var17_in_1y3 delta_num_trasp_var17_in_1y3
delta_imp_trasp_var17_out_1y3 delta_num_trasp_var17_out_1y3
delta_imp_trasp_var33_in_1y3 delta_num_trasp_var33_in_1y3
delta_imp_trasp_var33_out_1y3 delta_num_trasp_var33_out_1y3


In [7]:
for c in train.columns:
    cmin = train[c].min()
    cmax = train[c].max()
    test.loc[test[c] < cmin, c] = cmin
    test.loc[test[c] > cmax, c] = cmax

In [89]:
import xgboost as xgb

In [90]:
dtest = xgb.DMatrix(test)

In [91]:
params = {'alpha': .6,
          'colsample_bytree': .68,
          'eta': .03,
          'eval_metric': 'auc',
          'lambda': 1.,
          'max_depth': 3,
          'min_child_weight': 2.0,
          'objective': 'binary:logistic',
          'seed': 0,
          'silent': 1,
          'subsample': .55}

In [92]:
preds_all = list()
for seed in range(50):
    print(seed, flush=True)
    np.random.seed(seed)
    train.loc[:, 'target'] = target
    ind = np.random.choice(train.index[target == 0], size=target.sum(), replace=False)
    subs = pd.concat((train.loc[ind, :], train.loc[target==1, :]))
    newtarg = subs.target.values.copy()
    subs.drop('target', axis=1, inplace=True)
    dtrain = xgb.DMatrix(subs, newtarg)
    
    gbm = xgb.train(params, dtrain, num_boost_round=330, verbose_eval=False)
    
    preds = gbm.predict(dtest)
    
    preds_all.append(preds)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [98]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = np.array(preds_all).mean(axis=0)
sample.to_csv('submission/xgb_usmp_bag50.csv', index_label='ID')

In [ ]:
ans = xgb.cv(params, dtrain, num_boost_round=500, nfold=5, stratified=True, verbose_eval=True)

In [53]:
print(ans['test-auc-mean'].max())

0.8285768


In [56]:
r = ans[ans['test-auc-mean'] == ans['test-auc-mean'].max()]

In [46]:
from hyperopt import hp, tpe, fmin, STATUS_OK

In [63]:
space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'subsample': hp.uniform('subsample', .1, 1.),
    'colsample_bytree': hp.uniform('colsample_bytree', .1, 1.),
    'min_child_weight': hp.quniform('min_child_weight', 1, 8, 1),
    'lambda': hp.uniform('lambda', 0., 1.),
    'alpha': hp.uniform('alpha', 0., 1.)
}

In [66]:
from pprint import pprint

In [75]:
from libtelepot import sendMessage

In [81]:
def objective(space):
    lmb = np.exp(space['lambda']*10)
    
    params = {'objective': 'binary:logistic', 
              'eval_metric': 'auc',
              'eta': 0.1,
              'max_depth': int(space['max_depth']),
              'subsample': space['subsample'],
              'colsample_bytree': space['colsample_bytree'],
              'min_child_weight': space['min_child_weight'],
              'lambda': lmb,
              'alpha': space['alpha'],
              'silent': 1,
              'seed': 0}
    
    pprint(params)
    
    ans = xgb.cv(params, dtrain, num_boost_round=200, nfold=5, stratified=True)#, verbose_eval=True)
    
    print(ans[ans['test-auc-mean'] == ans['test-auc-mean'].max()], flush=True)
    r = ans[ans['test-auc-mean'] == ans['test-auc-mean'].max()]
    
    sendMessage(str(r))
    
    return {'loss': 1-r['test-auc-mean'].values[0], 'loss_variance': r['test-auc-std'].values[0],
            'status': STATUS_OK}

In [ ]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=50)

In [84]:
objective(best)

{'alpha': 0.59523951469438385,
 'colsample_bytree': 0.6716421365239873,
 'eta': 0.1,
 'eval_metric': 'auc',
 'lambda': 1.0487535673720274,
 'max_depth': 3,
 'min_child_weight': 2.0,
 'objective': 'binary:logistic',
 'seed': 0,
 'silent': 1,
 'subsample': 0.54113696671982725}
    test-auc-mean  test-auc-std  train-auc-mean  train-auc-std
88       0.836473      0.016721        0.865616       0.003466


{'loss': 0.16352659999999997,
 'loss_variance': 0.01672130782684177,
 'status': 'ok'}

In [93]:
1-0.16352659999999997

0.8364734

In [ ]:
preds_all = list()
for seed in range(50):
    print(seed, flush=True)
    params['seed'] = seed
    gbm = xgb.train(params, dtrain, num_boost_round=560)
    preds = gbm.predict(dtest)
    preds_all.append(preds)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [26]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = np.array(preds_all).mean(axis=0)
sample.to_csv('submission/init_xgb.csv', index_label='ID')